In [4]:
import tensorflow as tf
from transformers import TFWhisperForConditionalGeneration
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


# 1. 모델 디렉토리 설정
# 이 디렉토리에는 config.json, pytorch_model.bin, generation_config.json,
# tokenizer_config.json, special_tokens_map.json, preprocessor_config.json, normalizer.json, merges.txt, vocab.json, added_tokens.json 등이 포함되어 있어야 합니다.
model_dir = "whisper-large-v3"  # 사용자의 모델 디렉토리 경로로 변경

# 2. PyTorch 체크포인트를 기반으로 TensorFlow 모델로 변환 (from_pt=True)
tf_model = TFWhisperForConditionalGeneration.from_pretrained(model_dir, from_pt=True)
print("TensorFlow 모델이 로드되었습니다.")

# 3. SavedModel 형식으로 저장 (TFLite 변환에 필요)
saved_model_dir = "./tf_whisper_saved"
tf.saved_model.save(tf_model, saved_model_dir)
print("SavedModel이 저장되었습니다:", saved_model_dir)

# 4. TFLite 변환
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

# 최적화를 활성화하여 모델 경량화 (포스트 트레이닝 양자화 등)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# (선택 사항) 대표 데이터셋을 제공하여 양자화 보정을 진행할 수 있습니다.
# 아래는 대표 데이터셋 함수의 예시입니다. 실제 데이터셋을 사용하면 더 좋습니다.
# def representative_dataset():
#     for _ in range(100):
#         # preprocessor_config.json에 따라 입력은 (1, 128, 3000) 형태로 예상됨
#         dummy_input = np.random.rand(1, 128, 3000).astype(np.float32)
#         yield [dummy_input]
# converter.representative_dataset = representative_dataset

tflite_model = converter.convert()

# 5. TFLite 모델 파일로 저장
tflite_model_file = "whisper-large-v3.tflite"
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

print("TFLite 모델이 성공적으로 저장되었습니다:", tflite_model_file)


E0000 00:00:1743953106.139622 1883371 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2025-04-06 15:25:06.141116: W tensorflow/core/framework/op_kernel.cc:1857] OP_REQUIRES failed at conv_ops_impl.h:1204 : INVALID_ARGUMENT: No DNN in stream executor.


InvalidArgumentError: Exception encountered when calling layer 'conv1' (type Conv1D).

{{function_node __wrapped__Conv2D_device_/job:localhost/replica:0/task:0/device:GPU:0}} No DNN in stream executor. [Op:Conv2D] name: 

Call arguments received by layer 'conv1' (type Conv1D):
  • inputs=tf.Tensor(shape=(1, 3001, 128), dtype=float32)